In [27]:
import gc
import math
import os
import random
import pathlib
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split

import lightgbm as lgb
import xgboost as xgb

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

In [2]:
%matplotlib inline

In [106]:
def add_features(
        df,
        first_index=None,
        last_index=None,
        sample_size=150000,
        holdout_size=50000,
        smootch_windows_size = (3, 5, 7)
    ):
    
    print("df.shape:", df.shape)
    print("df.index[:3]", df.index[:3])
    print("df.index[-3:]", df.index[-3:])
    if first_index == None or last_index == None:
        first_index = df.index.min()
        last_index = df.index.max()
    print("first_index: {}, last_index: {}".format(first_index, last_index))
    #sample_indexes = np.random.randint(first_index, last_index, sample_size)
    sample_indexes = random.sample(range(first_index, last_index + 1), sample_size)
    sample_indexes.sort()
    print("sample_indexes[:3]", sample_indexes[:3])
    print("sample_indexes[-3:]", sample_indexes[-3:])
    print("len(sample_indexes):", len(sample_indexes))
    #print("sample_indexes.shape:", sample_indexes.shape)    
    smootch_feature_names = ['smootch_mean_ws_{}'.format(window_size) for window_size in smootch_windows_size]
    #half_windows_size = [ws // 2 for ws in smootch_windows_size]

    acoustic_data_series = df['acoustic_data']
    
    sample_df = df.iloc[sample_indexes]
    sample_df.reset_index(inplace=True)
    sample_df.drop(columns=['index'], inplace=True) # There is need map sample_df.index -> sample_indexes
                                                    # Должно быть установелнно соотвествие индексов в sample_df
                                                    # (Где после .reset_index индексы - это
                                                    # все целые числа от 0 до sample_df.shape[0]) и sample_indexes -
                                                    # соответствующие индексы в df которые являються случайной
                                                    # выборкой из чисел о first_index до last_index и длинной 
                                                    # равной sample_df.index.shape[0] которая длинна равно sample_size 
                                                    # Соответсвенно, по видимому, in_window_begin_indexes и
                                                    # in_window_end_indexes надо вычислять как то по другому
    #print("just after create sample_df, sample_df.shape[0]:", sample_df.shape[0])
    
    #for feature_name in smootch_feature_names:
    #    sample_df[feature_name] = 0
    #begin_smootch_features_value = []
    #end_smootch_features_value = []
    
    #sample_indexes_set = set(sample_indexes)
    begin_indexes_set = set()
    end_indexes_set = set()
    
    start_time = time.time()
    #print("sample_df.shape[0] just before main loop:", sample_df.shape[0])
    sample_df_len = sample_df.shape[0]
    for window_size, feature_name in zip(smootch_windows_size, smootch_feature_names):
        
        print("\n" * 2)
        print("window_size:", window_size)
        print()
        feature_values_list = list(range(sample_size))
        print("sample_df.index.tolist()[:window_size]:\n", sample_df.index.tolist()[:window_size])
        print("df.index.tolist()[:window_size]:\n", df.index.tolist()[:window_size])

        print("sample_df.index.tolist()[-window_size:]:\n", sample_df.index.tolist()[-window_size:])
        print("df.index.tolist()[-window_size:]:\n", df.index.tolist()[-window_size:])
        
        half_window_size = window_size // 2
        ##>begin_indexes = sample_indexes[:half_window_size]  
        ##>print("begin_indexes:\n", begin_indexes)
        sample_begin_indexes = sample_indexes[:half_window_size] #? sample_df.index[:half_window_size]
        print("sample_begin_indexes:\n", sample_begin_indexes)
        #full_data_begin_indexes = df.index[:half_window_size].tolist() #df.index[sample_indexes[i]]
        full_data_begin_indexes = set(df.index[sample_begin_indexes].tolist())
        print("full_data_begin_indexes:\n", full_data_begin_indexes)
        
        #in_window_begin_indexes = [idx for idx in sample_begin_indexes if idx in full_data_begin_indexes]
        min_begin_index = min(full_data_begin_indexes)
        print("set(range(min_begin_index, min_begin_index + half_window_size))", set(range(min_begin_index, min_begin_index + half_window_size)))
        in_window_begin_indexes = full_data_begin_indexes.intersection(
            set(range(min_begin_index, min_begin_index + half_window_size))
        )
        print("in_window_begin_indexes:\n", in_window_begin_indexes)
        
        sample_end_indexes = sample_indexes[-half_window_size:]
        print("sample_end_indexes:\n", sample_end_indexes)

        #full_data_end_indexes = df.index[-half_window_size:].tolist()
        full_data_end_indexes = set(df.index[sample_end_indexes].tolist())
        print("full_data_end_indexes:", full_data_end_indexes)
        
        #in_window_end_indexes = [idx for idx in sample_end_indexes if idx in full_data_end_indexes]
        max_end_index = max(full_data_end_indexes)
        print("max_end_index:", max_end_index)
        print("max_end_index - half_window_size:", max_end_index - half_window_size)
        print("set(range(max_end_index - half_window_size, max_end_index + 1)):\n", set(range(max_end_index + 1 - half_window_size, max_end_index + 1)))
        in_window_end_indexes = full_data_end_indexes.intersection(
            set(range(max_end_index + 1 - half_window_size, max_end_index + 1))
        )
        print("in_window_end_indexes:\n", in_window_end_indexes)
        
        ##>begin_indexes = begin_indexes[begin_indexes <= half_window_size]
        ##>print("2 begin_indexes? :\n", begin_indexes)
        
        ##>end_indexes = sample_indexes[-half_window_size:]
        ##>print("2 end_indexes? :\n", end_indexes)
        print("in_window_begin_indexes:\n", in_window_begin_indexes)
        print("len(in_window_begin_indexes)", len(in_window_begin_indexes))
        print()
        if in_window_begin_indexes:
            begin_indexes_set = begin_indexes_set.union(in_window_begin_indexes)
            for i, b_idx in enumerate(sorted(tuple(in_window_begin_indexes))):
                print("i: {}, b_idx {}:".format(i, b_idx))
                print("type(b_idx):", type(b_idx))
                print("df[:window_size]:\n", df[:window_size])
                #sample_df[feature_name].iloc[b_idx] = df[b_idx]['acoustic_data'] - df.iloc[first_index:first_index + window_size]['acoustic_data'].mean()
                value = df.iloc[b_idx]['acoustic_data']
                print("begin 1 value:", value)
                print("df.iloc[first_index:first_index + window_size]['acoustic_data']:", df.iloc[first_index:first_index + window_size]['acoustic_data'])
                temp = df.iloc[first_index:first_index + window_size]['acoustic_data'].mean()
                print("begin temp:", temp)
                value = value - temp
                print("begin 2 value:", value)
                feature_values_list[i] = value
        print("in_window_end_indexes:\n", in_window_end_indexes)
        print("len(in_window_end_indexes)", len(in_window_end_indexes))
        print()
        if in_window_end_indexes:
            end_indexes_set = end_indexes_set.union(in_window_end_indexes)
            for i, e_idx in enumerate(sorted(tuple(in_window_end_indexes))):
                print("i: {}, e_idx {}:".format(i, e_idx))
                print("type(e_idx):", type(e_idx))
                print("df[-window_size:]:\n", df[-window_size:])
                #sample_df[feature_name].iloc[e_idx] = df[e_idx]['acoustic_data'] - df.iloc[last_index - window_size:]['acoustic_data'].mean()
                value = df.iloc[e_idx]['acoustic_data']
                print("end 1 value:", value)
                print("df.iloc[last_index + 1 - window_size:]['acoustic_data']:\n", df.iloc[last_index + 1 - window_size:]['acoustic_data'])
                temp = df.iloc[last_index + 1 - window_size:]['acoustic_data'].mean()
                print("end temp:", temp)
                value = value - temp
                print("end 2 value:", value)
                feature_values_list[last_index - i] = value
        #print("in main loop, sample_df.shape:", sample_df.shape)
        print("sample_df_len:", sample_df_len)
        print("begin_indexes_set:\n", begin_indexes_set)
        print("len(begin_indexes_set):", len(begin_indexes_set))
        print("begin_indexes_set:\n", end_indexes_set)
        print("len(begin_indexes_set):", len(end_indexes_set))
        first_regular_idx = len(begin_indexes_set)
        last_regular_idx = sample_df_len - len(end_indexes_set)
        print("first_regular_idx:", first_regular_idx)
        print("last_regular_idx:", last_regular_idx)
        #for i in range(sample_df_len)[slice_begin:slice_end]:
        for i in range(first_regular_idx, last_regular_idx):
            sample_idx = sample_indexes[i]
            feature_values_list[i] = acoustic_data_series.iloc[
                sample_idx - half_window_size:sample_idx + half_window_size
            ].mean()
        #print("sample_df.shape[0] before assign feature_values_list:", sample_df.shape[0])
        #print("len(feature_values_list):", len(feature_values_list))
        sample_df[feature_name] = feature_values_list
        
    #sample_df_indexes_set = set(sample_df.index)
    #train_indexes = sample_df_indexes_set
    holdout_df = None
    if holdout_size > 0:
        holdout_indexes = np.random.randint(0, sample_df.shape[0], holdout_size)
        #print("sample_df.index:\n", sample_df.index.tolist())
        #print("holdout_indexes:\n", holdout_indexes)
        #train_indexes = np.array(sample_df.index)
        holdout_df = sample_df.iloc[holdout_indexes]
        holdout_df.reset_index(inplace=True)
        holdout_df.drop(columns=['index'], inplace=True)
    
        #train_indexes = sorted(tuple(sample_df_indexes_set.difference(set(holdout_indexes))))
        train_indexes = sorted(tuple(set(sample_df.index).difference(set(holdout_indexes))))
        #print("train_indexes:\n", train_indexes)
        sample_df = sample_df.iloc[train_indexes]
        sample_df.reset_index(inplace=True)
        sample_df.drop(columns=['index'], inplace=True)
    print("Full calculation feature value time (with slicing) {} min:".format((time.time() - start_time) / 60))
    return sample_df, holdout_df

In [112]:
def add_features(
        df,
        first_index=None,
        last_index=None,
        sample_size=150000,
        holdout_size=50000,
        smootch_windows_size = (3, 5, 7)
    ):
    
    print("df.shape:", df.shape)
    print("df.index[:3]", df.index[:3])
    print("df.index[-3:]", df.index[-3:])
    if first_index == None or last_index == None:
        first_index = df.index.min()
        last_index = df.index.max()
    print("first_index: {}, last_index: {}".format(first_index, last_index))
    #sample_indexes = np.random.randint(first_index, last_index, sample_size)
    sample_indexes = random.sample(range(first_index, last_index + 1), sample_size)
    sample_indexes.sort()
    print("sample_indexes[:3]", sample_indexes[:3])
    print("sample_indexes[-3:]", sample_indexes[-3:])
    print("len(sample_indexes):", len(sample_indexes))
    #print("sample_indexes.shape:", sample_indexes.shape)    
    smootch_feature_names = ['smootch_mean_ws_{}'.format(window_size) for window_size in smootch_windows_size]
    #half_windows_size = [ws // 2 for ws in smootch_windows_size]

    acoustic_data_series = df['acoustic_data']
    
    sample_df = df.iloc[sample_indexes]
    sample_df.reset_index(inplace=True)
    sample_df.drop(columns=['index'], inplace=True) # There is need map sample_df.index -> sample_indexes
                                                    # Должно быть установелнно соотвествие индексов в sample_df
                                                    # (Где после .reset_index индексы - это
                                                    # все целые числа от 0 до sample_df.shape[0]) и sample_indexes -
                                                    # соответствующие индексы в df которые являються случайной
                                                    # выборкой из чисел о first_index до last_index и длинной 
                                                    # равной sample_df.index.shape[0] которая длинна равно sample_size 
                                                    # Соответсвенно, по видимому, in_window_begin_indexes и
                                                    # in_window_end_indexes надо вычислять как то по другому
    #print("just after create sample_df, sample_df.shape[0]:", sample_df.shape[0])
    
    #for feature_name in smootch_feature_names:
    #    sample_df[feature_name] = 0
    #begin_smootch_features_value = []
    #end_smootch_features_value = []
    
    #sample_indexes_set = set(sample_indexes)
    begin_indexes_set = set()
    end_indexes_set = set()
    
    start_time = time.time()
    #print("sample_df.shape[0] just before main loop:", sample_df.shape[0])
    sample_df_len = sample_df.shape[0]
    for window_size, feature_name in zip(smootch_windows_size, smootch_feature_names):
        
        print("\n" * 2)
        print("window_size:", window_size)
        print()
        feature_values_list = list(range(sample_size))

        half_window_size = window_size // 2

        sample_begin_indexes = sample_indexes[:half_window_size] #? sample_df.index[:half_window_size]

        full_data_begin_indexes = set(df.index[sample_begin_indexes].tolist())

        min_begin_index = min(full_data_begin_indexes)
        
        in_window_begin_indexes = full_data_begin_indexes.intersection(
            set(range(min_begin_index, min_begin_index + half_window_size))
        )
        print("in_window_begin_indexes:\n", in_window_begin_indexes)
        
        sample_end_indexes = sample_indexes[-half_window_size:]

        full_data_end_indexes = set(df.index[sample_end_indexes].tolist())
        
        max_end_index = max(full_data_end_indexes)
        print("max_end_index:", max_end_index)
        print("max_end_index - half_window_size:", max_end_index - half_window_size)

        in_window_end_indexes = full_data_end_indexes.intersection(
            set(range(max_end_index + 1 - half_window_size, max_end_index + 1))
        )
        print("in_window_begin_indexes:\n", in_window_begin_indexes)
        print("len(in_window_begin_indexes)", len(in_window_begin_indexes))
        print("in_window_end_indexes:\n", in_window_end_indexes)
        print("len(in_window_end_indexes):", len(in_window_end_indexes))

        print()
        if in_window_begin_indexes:
            begin_indexes_set = begin_indexes_set.union(in_window_begin_indexes)
            for i, b_idx in enumerate(sorted(tuple(in_window_begin_indexes))):
                print("i: {}, b_idx {}:".format(i, b_idx))
                print("type(b_idx):", type(b_idx))
                print("df[:window_size]:\n", df[:window_size])
                #sample_df[feature_name].iloc[b_idx] = df[b_idx]['acoustic_data'] - df.iloc[first_index:first_index + window_size]['acoustic_data'].mean()
                value = df.iloc[b_idx]['acoustic_data']
                print("begin 1 value:", value)
                print("df.iloc[first_index:first_index + window_size]['acoustic_data']:\n", df.iloc[first_index:first_index + window_size]['acoustic_data'])
                temp = df.iloc[first_index:first_index + window_size]['acoustic_data'].mean()
                print("begin temp:", temp)
                value = value - temp
                print("begin 2 value:", value)
                feature_values_list[i] = value
        if in_window_end_indexes:
            end_indexes_set = end_indexes_set.union(in_window_end_indexes)
            for i, e_idx in enumerate(sorted(tuple(in_window_end_indexes))):
                print("i: {}, e_idx {}:".format(i, e_idx))
                print("type(e_idx):", type(e_idx))
                print("df[-window_size:]:\n", df[-window_size:])
                #sample_df[feature_name].iloc[e_idx] = df[e_idx]['acoustic_data'] - df.iloc[last_index - window_size:]['acoustic_data'].mean()
                value = df.iloc[e_idx]['acoustic_data']
                print("end 1 value:", value)
                print("df.iloc[last_index + 1 - window_size:]['acoustic_data']:\n", df.iloc[last_index + 1 - window_size:]['acoustic_data'])
                temp = df.iloc[last_index + 1 - window_size:]['acoustic_data'].mean()
                print("end temp:", temp)
                value = value - temp
                print("end 2 value:", value)
                feature_values_list[last_index - i] = value
        #print("in main loop, sample_df.shape:", sample_df.shape)
        print("sample_df_len:", sample_df_len)
        print("begin_indexes_set:\n", begin_indexes_set)
        print("len(begin_indexes_set):", len(begin_indexes_set))
        print("begin_indexes_set:\n", end_indexes_set)
        print("len(begin_indexes_set):", len(end_indexes_set))
        first_regular_idx = len(begin_indexes_set)
        last_regular_idx = sample_df_len - len(end_indexes_set)
        print("first_regular_idx:", first_regular_idx)
        print("last_regular_idx:", last_regular_idx)
        #for i in range(sample_df_len)[slice_begin:slice_end]:
        for i in range(first_regular_idx, last_regular_idx):
            sample_idx = sample_indexes[i]
            feature_values_list[i] = acoustic_data_series.iloc[
                sample_idx - half_window_size:sample_idx + half_window_size
            ].mean()
        #print("sample_df.shape[0] before assign feature_values_list:", sample_df.shape[0])
        #print("len(feature_values_list):", len(feature_values_list))
        sample_df[feature_name] = feature_values_list
        
    #sample_df_indexes_set = set(sample_df.index)
    #train_indexes = sample_df_indexes_set
    holdout_df = None
    if holdout_size > 0:
        holdout_indexes = np.random.randint(0, sample_df.shape[0], holdout_size)
        #print("sample_df.index:\n", sample_df.index.tolist())
        #print("holdout_indexes:\n", holdout_indexes)
        #train_indexes = np.array(sample_df.index)
        holdout_df = sample_df.iloc[holdout_indexes]
        holdout_df.reset_index(inplace=True)
        holdout_df.drop(columns=['index'], inplace=True)
    
        #train_indexes = sorted(tuple(sample_df_indexes_set.difference(set(holdout_indexes))))
        train_indexes = sorted(tuple(set(sample_df.index).difference(set(holdout_indexes))))
        #print("train_indexes:\n", train_indexes)
        sample_df = sample_df.iloc[train_indexes]
        sample_df.reset_index(inplace=True)
        sample_df.drop(columns=['index'], inplace=True)
    print("Full calculation feature value time (with slicing) {} min:".format((time.time() - start_time) / 60))
    return sample_df, holdout_df

In [203]:
def add_features(
        df,
        input_first_index=None,
        input_last_index=None,
        sample_size=150000,
        holdout_size=50000,
        smootch_windows_size = (3, 5, 7)
    ):
    if input_first_index == None or input_last_index == None:
        input_first_index = df.index.min()
        input_last_index = df.index.max()
        
    sample_indexes = random.sample(range(input_first_index, input_last_index + 1), sample_size)
    sample_indexes.sort()
    print("sample_indexes[:7]:\n", sample_indexes[:7])
    print("sample_indexes[-7:]:\n", sample_indexes[-7:])
    #sample_first_index = min(sample_indexes)
    #sample_last_index = max(sample_indexes)
    #print("sample_first_index:", sample_first_index)
    #print("sample_last_index:", sample_last_index)
    
    smootch_feature_names = ['smootch_mean_ws_{}'.format(window_size) for window_size in smootch_windows_size]
    acoustic_data_series = df['acoustic_data']
    full_data_indexes = tuple(acoustic_data_series.index.tolist())
    '''
    print("acoustic_data_series.index[:3]:\n", acoustic_data_series.index[:3].tolist())
    print("acoustic_data_series.index[-3:]:\n", acoustic_data_series.index[-3:].tolist())
    '''
    
    print("acoustic_data_series[:7]:\n", acoustic_data_series[:7])
    print("acoustic_data_series[-7:]:\n", acoustic_data_series[-7:])
    
    '''
    print("df[:3]['acousitc_data']:\n", df[:3]['acoustic_data'])
    print("df[-3:]['acoustic_data']:\n", df[-3:]['acoustic_data'])
    '''
    
    sample_df = df.iloc[sample_indexes]
    '''
    print("before reset sample_df.index[:3]", sample_df.index[:3])
    print("before reset sample_df.index[-3:]", sample_df.index[-3:])
    print("before reset sample_df[:3]", sample_df[:3])
    print("before reset sample_df[-3:]", sample_df[-3:])
    '''
    sample_df.reset_index(inplace=True)
    sample_df.drop(columns=['index'], inplace=True)
    '''
    print("after reset sample_df.index[:3]", sample_df.index[:3])
    print("after reset sample_df.index[-3:]", sample_df.index[-3:])
    print("after reset sample_df[:3]", sample_df[:3])
    print("after reset sample_df[-3:]", sample_df[-3:])
    '''
    output_first_index = 0
    output_last_index = len(sample_df) - 1
    
    begin_indexes_set = set()
    end_indexes_set = set()
    
    start_time = time.time()
   
    sample_df_len = sample_df.shape[0]
    for window_size, feature_name in zip(smootch_windows_size, smootch_feature_names):
        print("\n\nwindow_size: {} feature_name: {}".format(window_size, feature_name))

        feature_values_list = list(range(sample_size))

        half_window_size = window_size // 2

        sample_begin_indexes = sample_indexes[:half_window_size] #? sample_df.index[:half_window_size]
        full_data_begin_indexes = set(df.index[sample_begin_indexes].tolist())
        min_full_data_index = min(full_data_indexes)
        in_window_full_data_begin_indexes = set(range(min_full_data_index, min_full_data_index + half_window_size))
        print("+++ in_window_full_data_begin_indexes:\n", in_window_full_data_begin_indexes)
              
        in_window_begin_indexes = full_data_begin_indexes.intersection(
            #set(range(min_begin_index, min_begin_index + half_window_size))
            #set(range(min_full_data_index, min_full_data_index + half_window_size))
            in_window_full_data_begin_indexes
        )
        print("---in_window_begin_indexes:\n", in_window_begin_indexes)
        
        sample_end_indexes = sample_indexes[-half_window_size:]
        full_data_end_indexes = set(df.index[sample_end_indexes].tolist())
        max_full_data_index = max(full_data_end_indexes)
        in_window_full_data_end_indexes = set(range(max_full_data_index + 1 - half_window_size, max_full_data_index + 1))
        print("+++ in_window_full_data_end_indexes:\n", in_window_full_data_end_indexes)
        
        in_window_end_indexes = full_data_end_indexes.intersection(
            #set(range(max_end_index + 1 - half_window_size, max_end_index + 1))
            #set(range(max_full_data_index + 1 - half_window_size, max_full_data_index))
            in_window_full_data_end_indexes
        )
        print("---in_window_end_indexes:\n", in_window_end_indexes)
        if in_window_begin_indexes:
            print("\nin_window_begin_indexes")
            print("sorted(tuple(in_window_begin_indexes)):\n", sorted(tuple(in_window_begin_indexes)))
            begin_indexes_set = begin_indexes_set.union(in_window_begin_indexes)
            for i, b_idx in enumerate(sorted(tuple(in_window_begin_indexes))):
                print("\ni:", i)
                #value = df.iloc[b_idx]['acoustic_data']
                print("sample_df value:", sample_df.iloc[i]['acoustic_data'])
                #value = acoustic_data_series.iloc[b_idx]
                value = sample_df.iloc[i]['acoustic_data']
                print("1 value:", value)
                #temp = df.iloc[input_first_index:input_first_index + window_size]['acoustic_data'].mean()
                print("acoustic_data_series.iloc[input_first_index:input_first_index + window_size]:\n",
                      acoustic_data_series.iloc[input_first_index:input_first_index + window_size]
                )
                temp = acoustic_data_series.iloc[input_first_index:input_first_index + window_size].mean()
                print("temp:", temp)
                value = value - temp
                print("2 value:", value)
                print("output_first_index + i:", output_first_index + i)
                feature_values_list[output_first_index + i] = value
                
        if in_window_end_indexes:
            print("\nin_window_end_indexes")
            print("sorted(tuple(in_window_end_indexes)):\n", sorted(tuple(in_window_end_indexes)))
            end_indexes_set = end_indexes_set.union(in_window_end_indexes)
            for i, e_idx in enumerate(sorted(tuple(in_window_end_indexes))):
                print("\ni:", i)
                #value = df.iloc[e_idx]['acoustic_data']
                print("sample_df value:", sample_df.iloc[output_last_index - i]['acoustic_data'])
                value = acoustic_data_series.iloc[e_idx]
                #value = sample_df.iloc[output_last_index - i]['acoustic_data']
                print("1 value:", value)
                #temp = df.iloc[input_last_index + 1 - window_size:]['acoustic_data'].mean()
                print("acoustic_data_series.iloc[input_last_index + 1 - window_size:]:\n",
                      acoustic_data_series.iloc[input_last_index + 1 - window_size:]
                )
                temp = acoustic_data_series.iloc[input_last_index + 1 - window_size:].mean()
                print("temp:", temp)
                value = value - temp
                print("2 value:", value)
                print("output_last_index - 1 - i:", output_last_index - 1 - i)
                feature_values_list[output_last_index - i] = value
                
        first_regular_idx = len(begin_indexes_set)
        last_regular_idx = sample_df_len - len(end_indexes_set)
        for i in range(first_regular_idx, last_regular_idx):
            sample_idx = sample_indexes[i]
            feature_values_list[i] = acoustic_data_series.iloc[
                sample_idx - half_window_size:sample_idx + half_window_size
            ].mean()
        sample_df[feature_name] = feature_values_list
        
    holdout_df = None
    if holdout_size > 0:
        holdout_indexes = np.random.randint(0, sample_df.shape[0], holdout_size)
        holdout_df = sample_df.iloc[holdout_indexes]
        holdout_df.reset_index(inplace=True)
        holdout_df.drop(columns=['index'], inplace=True)
        train_indexes = sorted(tuple(set(sample_df.index).difference(set(holdout_indexes))))
        sample_df = sample_df.iloc[train_indexes]
        sample_df.reset_index(inplace=True)
        sample_df.drop(columns=['index'], inplace=True)
    print("Full calculation feature value time (with slicing) {} min:".format((time.time() - start_time) / 60))
    return sample_df, holdout_df

In [6]:
not_seen_data_df = pd.read_csv(
    '../input/train/train.csv',
    #nrows=100000000,
    names=['acoustic_data', 'time_to_failure'],
    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32},
    skiprows=1,
    nrows=5656572
)

In [204]:
%%time
featured_not_seen_data_df, _ = add_features(
    not_seen_data_df[:1000],
    sample_size=not_seen_data_df[:1000].shape[0],
    holdout_size=0
)

sample_indexes[:7]:
 [0, 1, 2, 3, 4, 5, 6]
sample_indexes[-7:]:
 [993, 994, 995, 996, 997, 998, 999]
acoustic_data_series[:7]:
 0    12.0
1     6.0
2     8.0
3     5.0
4     8.0
5     8.0
6     9.0
Name: acoustic_data, dtype: float32
acoustic_data_series[-7:]:
 993    6.0
994    7.0
995    8.0
996    9.0
997    7.0
998    8.0
999    5.0
Name: acoustic_data, dtype: float32


window_size: 3 feature_name: smootch_mean_ws_3
+++ in_window_full_data_begin_indexes:
 {0}
---in_window_begin_indexes:
 {0}
+++ in_window_full_data_end_indexes:
 {999}
---in_window_end_indexes:
 {999}

in_window_begin_indexes
sorted(tuple(in_window_begin_indexes)):
 [0]

i: 0
sample_df value: 12.0
1 value: 12.0
acoustic_data_series.iloc[input_first_index:input_first_index + window_size]:
 0    12.0
1     6.0
2     8.0
Name: acoustic_data, dtype: float32
temp: 8.666667
2 value: 3.333333
output_first_index + i: 0

in_window_end_indexes
sorted(tuple(in_window_end_indexes)):
 [999]

i: 0
sample_df value: 5.0
1 value: 5.

In [177]:
featured_not_seen_data_df.shape

(1000, 5)

In [181]:
featured_not_seen_data_df['acoustic_data'][:7].values

array([ 5.,  8.,  9.,  7., -5.,  3.,  5.], dtype=float32)

In [187]:
featured_not_seen_data_df.head(10)

,acoustic_data,time_to_failure,smootch_mean_ws_3,smootch_mean_ws_5,smootch_mean_ws_7
0,8.0,1.4691,6.5,7.25,7.333333
1,7.0,1.4691,8.0,4.75,5.000000
2,5.0,1.4691,4.0,1.25,2.333333
3,2.0,1.4691,3.5,3.00,1.666667
4,2.0,1.4691,2.0,3.00,2.333333
5,4.0,1.4691,5.0,4.50,3.666667
6,5.0,1.4691,4.0,4.00,4.000000
7,6.0,1.4691,4.0,4.75,5.166667
8,7.0,1.4691,7.0,7.00,7.333333
9,8.0,1.4691,7.5,9.00,8.500000


In [182]:
featured_not_seen_data_df['acoustic_data'][-7:].values

array([10.,  7.,  3.,  3.,  2., -1., -4.], dtype=float32)

In [188]:
featured_not_seen_data_df.tail(10)

,acoustic_data,time_to_failure,smootch_mean_ws_3,smootch_mean_ws_5,smootch_mean_ws_7
990,0.0,1.469098,0.5,2.25,3.166667
991,3.0,1.469098,3.0,2.25,2.166667
992,3.0,1.469098,3.0,3.00,3.000000
993,6.0,1.469098,4.5,5.50,5.666667
994,9.0,1.469098,9.5,8.75,7.500000
995,7.0,1.469098,8.5,6.50,6.500000
996,3.0,1.469098,1.5,3.25,4.000000
997,3.0,1.469098,3.0,1.75,2.500000
998,2.0,1.469098,1.5,1.00,1.000000
999,-4.0,1.469098,-2.0,-3.20,-3.714286


In [59]:
not_seen_data_df[:1000].shape

(1000, 2)

In [60]:
not_seen_data_df[:1000].index.max()

999

In [26]:
np.random.

In [30]:
random.

In [36]:
np.random.rand?

In [37]:
np.random.rand(20)

array([0.94842548, 0.15565242, 0.13225282, 0.44878307, 0.79942495,
       0.46168806, 0.16916164, 0.35274032, 0.16233877, 0.06494294,
       0.4119373 , 0.8498234 , 0.89188705, 0.86023744, 0.99186619,
       0.57907303, 0.92444036, 0.80596842, 0.35860679, 0.09529798])

In [42]:
random.sample?

In [39]:
np.random.randint?

In [49]:
random.sample(range(10), 10)

[3, 8, 0, 5, 4, 2, 7, 6, 9, 1]

In [56]:
np.random.sample?

In [87]:
temp_df = not_seen_data_df[:1000]

In [91]:
temp_df[-7:]

,acoustic_data,time_to_failure
993,6.0,1.469099
994,7.0,1.469099
995,8.0,1.469099
996,9.0,1.469099
997,7.0,1.469099
998,8.0,1.469099
999,5.0,1.469099


In [96]:
first_idx = temp_df.index.min()
last_idx = temp_df.index.max()

In [97]:
print(first_idx, last_idx)

0 999


In [100]:
ws = 5

In [104]:
print(first_idx + ws)
print(last_idx - ws)

5
994


In [101]:
temp_df[first_idx:first_idx + ws]['acoustic_data']

0    12.0
1     6.0
2     8.0
3     5.0
4     8.0
Name: acoustic_data, dtype: float32

In [103]:
temp_df[last_idx - ws:last_idx]['acoustic_data']

994    7.0
995    8.0
996    9.0
997    7.0
998    8.0
Name: acoustic_data, dtype: float32

In [105]:
temp_df[last_idx + 1 - ws:]['acoustic_data']

995    8.0
996    9.0
997    7.0
998    8.0
999    5.0
Name: acoustic_data, dtype: float32